In [ ]:
import pathlib
from pathlib import Path
import json
import pandas as pd

In [ ]:
path = pathlib.Path("/labeling.json")

In [ ]:
corrected_labeling = json.load(open(path, 'rb'))

In [ ]:
filename = []
text = []
labels = []
for image in corrected_labeling:     
    file_name = image["meta"]["file"]
    for element in image["annotations"]:
        for y in element["result"]:
#             print(file_name, y["value"]["text"], y["value"]["labels"])
            filename.append(file_name)
            text.append(y["value"]["text"])
            labels.append(y["value"]["labels"])

In [ ]:
ner_df = pd.DataFrame()

In [ ]:
ner_df["Filename"] = filename
ner_df["Text"] = text
ner_df["Label"] = labels

In [ ]:
ner_labeling = pd.concat([ner_labeling, ner_df], ignore_index = True)

In [ ]:
ner_labeling

In [ ]:
import openpyxl
ner_labeling.to_excel('ner_labeling.xlsx')

## DeepPavlov

In [ ]:
# def Convert(string):
#     li = list(string.split("\n"))
#     return li

In [ ]:
import deeppavlov

In [ ]:
from deeppavlov import build_model
#ner_model = build_model('ner_collection3_bert', download=True, install=True)

In [ ]:
texts = []
dp_dict = {}
filenames = []
for key, value in json_results.items():
    cleaned_list = [elem for elem in value if elem.strip()]
    value_clean = [x for x in cleaned_list if x != '']
    if len(value_clean) == 0:
        pass
    else:
        result = ner_model(value_clean)
        dp_dict[key] = (result[0], result[1])  
#     print(len(dp_dict))

In [ ]:
dp_dict

## SpaCy

In [ ]:
import spacy

In [ ]:
ru_model_large = spacy.load("ru_core_news_lg")

In [ ]:
import json
json_ner = json.load(open('raw_recognition/results.json', 'rb'))

In [ ]:
json_pdf = json.load(open('raw_recognition/results_pdf.json', 'rb'))

In [ ]:
def merge_two_dicts(x, y):
    """Given two dictionaries, merge them into a new dict as a shallow copy."""
    z = x.copy()
    z.update(y)
    return z

In [ ]:
json_results = merge_two_dicts(json_ner, json_pdf)

In [ ]:
len(json_results)

In [ ]:
for key, value in json_pdf.items():
    texts.append(" ".join(value))
    print(texts[0])

In [ ]:
def get_tag(text):
    output = []
    labels = []
    result = ru_model_large(text)
    for ent in result.ents:
        output.append(ent.text)
        labels.append(ent.label_)
    return output, labels


In [ ]:
texts = []
spacy_dict_pdf = {}
output = []
labels = []
for key, value in json_pdf.items():
    texts.append(" ".join(value))
    file_name = key
    for text in texts:
        output, labels = get_tag(text)
        spacy_dict_pdf[key] = (output, labels)
    print(len(spacy_dict_pdf))

In [ ]:
with open('spacy_ner_pdf.json', 'w') as fp:
    json.dump(spacy_dict_pdf, fp)

 ## Natasha

In [ ]:
from navec import Navec
from slovnet import NER

In [ ]:
text = 'Европейский союз добавил в санкционный список девять политических деятелей из самопровозглашенных республик Донбасса — Донецкой народной республики (ДНР) и Луганской народной республики (ЛНР) — в связи с прошедшими там выборами. Об этом говорится в документе, опубликованном в официальном журнале Евросоюза. В новом списке фигурирует Леонид Пасечник, который по итогам выборов стал главой ЛНР. Помимо него там присутствуют Владимир Бидевка и Денис Мирошниченко, председатели законодательных органов ДНР и ЛНР, а также Ольга Позднякова и Елена Кравченко, председатели ЦИК обеих республик. Выборы прошли в непризнанных республиках Донбасса 11 ноября. На них удержали лидерство действующие руководители и партии — Денис Пушилин и «Донецкая республика» в ДНР и Леонид Пасечник с движением «Мир Луганщине» в ЛНР. Президент Франции Эмманюэль Макрон и канцлер ФРГ Ангела Меркель после встречи с украинским лидером Петром Порошенко осудили проведение выборов, заявив, что они нелегитимны и «подрывают территориальную целостность и суверенитет Украины». Позже к осуждению присоединились США с обещаниями новых санкций для России.'

path = Navec.load('navec_news_v1_1B_250K_300d_100q.tar') # модель
ner = NER.load('slovnet_ner_news_v1.tar') # эмбеддинги
ner.navec(path)

In [ ]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,

    Doc
)

In [ ]:
segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

In [ ]:
def get_pred(text):
    output = []
    labels = []
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_ner(ner_tagger)
    for span in doc.spans:
        output.append(span.text)
        labels.append(span.type)
    return output, labels

In [ ]:
texts = []
natasha_dict = {}

for key, value in json_results.items():
    texts.append(" ".join(value))
    file_name = key
    for text in texts:
        output, labels = get_pred(text)

        natasha_dict[key] = (output, labels)
    print(len(natasha_dict))

In [ ]:
natasha_dict

In [ ]:
with open('deep_pavlov.json', 'w') as fp:
    json.dump(dp_dict, fp)